1. Create conda env with GPU
2. Create data generator with augmentation
3. Create simple model
4. create advanced model with Bayesian Optimization for Hyperparameter tuning

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import glob
%reload_ext autoreload
%autoreload 2



In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [7]:
def display_batch_of_images(batch):
  import matplotlib.pyplot as plt

  for i in range(len(batch[0])):

    image = batch[0][i][:,:,:,0]

    # normalize to [0,1]
    image =  (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-9)

    segmentation = tf.math.argmax(batch[1][i], axis=3) # undo to_categorical

    segmentation = tf.cast(segmentation, tf.float32)
    segmentation = tf.multiply((255.0 / NUM_CLASSES), segmentation)
    segmentation = segmentation / 255.0

    plt.figure()

    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Image')

    plt.subplot(1, 2, 2)
    plt.imshow(segmentation, vmin=0, vmax=1)
    plt.title('Ground Truth Segmentation')

    plt.show()

In [ ]:
import albumentations as A
from data_generator import SegmentationDataGenerator

BATCH_SIZE = 6
NUM_CLASSES = 34
IMAGE_SIZE = (256, 256)

aug = A.Compose([
        A.VerticalFlip(p=0.5),              
        A.RandomRotate90(p=0.5),
        A.PixelDropout(p=0.01),
        A.OneOf([
            A.MotionBlur(p=0.2),
            A.Blur(p=0.1),
        ], p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=20, p=0.2),
        A.OpticalDistortion(p=0.3),
        A.GridDistortion(p=.1),
        A.RandomBrightnessContrast(p=1)
    ]
)

train_data = SegmentationDataGenerator(
    image_directory = f"F:/dev/TSM_CompVis-Segmantic-Segmentation/data/train/train2/*_img_*",
    segmentation_directory = f"F:/dev/TSM_CompVis-Segmantic-Segmentation/data/train/train2/*_lbl_*",
    num_classes = NUM_CLASSES,
    batch_size = BATCH_SIZE,
    augmentation = aug
)
train_data_gen = train_data.get_data_generator()

val_data = SegmentationDataGenerator(
    image_directory = f"F:/dev/TSM_CompVis-Segmantic-Segmentation/data/val/*_img_*",
    segmentation_directory = f"F:/dev/TSM_CompVis-Segmantic-Segmentation/data/val/*_lbl_*",
    num_classes = NUM_CLASSES,
    batch_size = BATCH_SIZE,
    augmentation = aug
)
val_data_gen = val_data.get_data_generator()

In [ ]:
batch = train_data_gen.__next__()
display_batch_of_images(batch)

In [ ]:
batch = val_data_gen.__next__()
display_batch_of_images(batch)